## Notes
My attempt at making a new interactive notebook.  
Main focus:  
    - Switching anndata objects  
    - Finding DEGs, probably precalculated table  
    - Plotting options  
        - Dotplot, featureplot  

Will start by trying to overhaul my interactive plotting code.  
First objective will be plotting a umap and being able to change the anndata object.  

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import anndata as ad
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import warnings
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
ecto_interactive_nocycle = sc.read('h5ad_files/mouse/ecto_andrea/ecto_interactive_nocycle.h5ad')

In [ ]:
anndata_objects = {'ecto_interactive_nocycle':ecto_interactive_nocycle,}

In [ ]:
anndata_objects['E12.5'] = ecto_interactive_nocycle[ecto_interactive_nocycle.obs['sample']=='12'].copy()

In [ ]:
from dash import Dash, Input, Output, html, dcc
import pandas as pd


app = Dash(__name__)

app.layout = html.Div([
    html.Label('Select object'),
    dcc.Dropdown(
        id='anndata_var',
        options=list(anndata_objects)
    ),
    html.Label('Select gene'),
    dcc.Dropdown(
        id='gene_var',
    )]
)


@app.callback(
    Output('gene_var', 'options'),
    Input('anndata_var', 'value'),
    prevent_initial_call=True
)
def update_output(anndata_var):
    return anndata_objects[anndata_var].var.index.tolist()
    #return df[df.process == value].code


if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
app = Dash(__name__)


app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label("Select object"),
            dcc.Dropdown(options = list(anndata_objects), 
                         id="anndata",
                        ),
            html.Label("Select gene"),
            dcc.Dropdown(
                         id="dropdown_var",
                        ),
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
        ],
            style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Select obs"),
            dcc.Dropdown(
                         id="dropdown_obs",
                        ),
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),
    ],
        style={'padding': '10px 5px'}),
    html.Div([
      dcc.Graph(id="graph1"),
        
    ],
        style={'width': '49%', 'display': 'inline-block'}),
     html.Div([
      dcc.Graph(id="graph2")
    ],
        style={'width': '49%', 'display': 'inline-block'}),
    html.Div(id="cluster_select",
        style={'width': '49%', 'display': 'inline-block'},),
    html.Div(id='deg_table'),
    html.Div(dcc.Store(id='gene_plotted')),
])

##set anndata object for other menus
@app.callback(
    Output('dropdown_var', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].var.index.tolist()

@app.callback(
    Output('dropdown_obs', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].obs.columns.tolist()

@app.callback(
    Output("graph1", "figure"),
    Output('gene_plotted','data'),
    Input("slider_marker_size", "value"),
    Input('deg_table_callback', 'active_cell'),
    Input('anndata', 'value'),    
    Input("dropdown_var", "value"),
    Input('deg_table_callback', 'derived_viewport_data'),
)

def update_graph_1(size_var,active_cell,dropdown_var,data_deg,anndata):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    elif callback_id == 'deg_table_callback':
        try:
            color_var = data_deg[active_cell['row']]['names']
        except:
            color_var = dropdown_var
        
    else:
        color_var = dropdown_var

    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = anndata_objects[anndata][:,color_var].X.toarray().flatten(),
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        color_continuous_scale = plotly_reds,
                        labels = {'color':color_var},
                        width=700, height=700,
                       )
    fig.update_traces(marker={'size': size_var,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(uirevision='constant')
    return fig, color_var

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
####dotplot code
fig_buffer = BytesIO()

dp = sc.pl.rank_genes_groups_dotplot(anndata_objects['ecto_interactive_nocycle'], 
                                groupby = 'leiden1',
                                n_genes=4,
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-4,
    vmax=4,
    min_logfoldchange=2,
                                key = 'wilcoxon_leiden1',
    colorbar_title='log fold change',
                               return_fig = True)
dp.savefig(fig_buffer, format='png')
plt.close()
encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")

fig_src = "data:image/png;base64, " + encoded

In [ ]:
port = 11199
initial_gene = 'Car4'
initial_obs = 'sample'

app = Dash(__name__,suppress_callback_exceptions=True)


#####selection list grid
df_list = pd.DataFrame(list(anndata_objects['ecto_interactive_nocycle'].obs['leiden1'].cat.categories),columns=['order categories:'])
grid = dag.AgGrid(
    id="select_categories",
    rowData=df_list.to_dict("records"),
    columnDefs=[{"field": i,} for i in df_list.columns],
    dashGridOptions={
                "animateRows": True,
                "enableCellTextSelection": False,
                "ensureDomOrder": True,
                "rowDragManaged": True,
                #"rowDragMultiRow": True,
                "rowSelection": "multiple",
                "rowDragEntireRow": True,
                "pagination": False,
                "suppressRowClickSelection": True,
            },
)

##########################################################
######################  Display of tab 1 #################
##########################################################
tab1 = html.Div([
    html.Div([
        html.Div([
            
            
            ##############################################
            
            ##############################################
            ######  select gene to plot from list  #######
            ##############################################
            html.Label("Select gene"),
            dcc.Dropdown(
                         id="dropdown_var",
                        value = initial_gene,
                        ),
            ##############################################
            
            ##############################################
            ####other gene values to show if available####
            ##############################################
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            ##############################################
            
            ##############################################
            #######set dot size of both umap plots########
            ##############################################
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
            ##############################################
        ],
            style={'width': '49%', 'display': 'inline-block'}), #html style of left dropdown section

        
        html.Div([
            ##############################################
            ######  select obs to plot from list  #######
            ##############################################
            html.Label("Select obs"),
            dcc.Dropdown(
                         id="dropdown_obs",
                         value = initial_obs
                        ),
            ##############################################
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}), #html style of right dropdown section
        
    ],
        style={'padding': '10px 5px'}), #html style of dropdown section

    ##############################################
    ##########  plot left umap graph  ############
    ##############################################
    html.Div([
      dcc.Graph(id="graph1"),
        ],
        style={'width': '49%', 'display': 'inline-block'}),#html style of left graph section
    ##############################################
    
    ##############################################
    ##########  plot right umap graph  ###########
    ##############################################
    html.Div([
      dcc.Graph(id="graph2")
    ],
            style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),#html style of right graph section
    ##############################################
    
    html.Div(dcc.Store(id='gene_plotted')),
])


##########################################################
######################  Display of tab 2 #################
##########################################################
left_tab_sub_left = 
left_tab_sub_right = 

#components that span the entire left tab
html_drop_dp_obs = html.Div([html.Label("Select obs:"),
                                      dcc.Dropdown(
                                          id="drop_dp_obs",
                                          options = list(anndata_objects[anndata].obs.dtypes[anndata_objects[anndata].obs.dtypes =='category'].index),
                                      ),
                                     ],
                                    )

html_drop_dp_method = html.Div([html.Label("Select method:"),
                                         dcc.Dropdown(id="drop_dp_method",
                                                      options = ['logreg', 't-test', 'wilcoxon', 't-test_overestim_var'], 
                                                      value = 'wilcoxon',
                                                     ),
                                        ],
                                       )

left_tab = 


tab2 = html.Div([
                ###left tab
    html.Div([html.Label("Select obs:"),
              dcc.Dropdown(
                  id="dropdown_obs_dotplot",
                  options = list(anndata_objects[anndata].obs.dtypes[anndata_objects[anndata].obs.dtypes =='category'].index),
              ),
              html.Br(),
              html.Br(),
                           
              html.Div([html.Label("Select method:"),
                        dcc.Dropdown(id="dropdown_dotplot_method",
                                     options = ['logreg', 't-test', 'wilcoxon', 't-test_overestim_var'], 
                                     value = 'wilcoxon',
                                    ),
                        html.Br(),
                        html.Br(),
                           
                        html.Label("min fold change: "),
                        dcc.Input(
                            id = 'dotplot_min_log_fold_change',
                            type='number',
                            value=2.5,
                        ),
                        html.Br(),
                        html.Br(),
                           
                        html.Label("vmin: "),
                        dcc.Input(
                            id = 'dotplot_vmin',
                            type='number',
                            max = 0,
                            value=-4,
                        ),
                        html.Br(),
                        html.Br(),
                           
                        html.Label("vmax: "),
            dcc.Input(
                id = 'dotplot_vmax',
                type='number',
                min = 0,
                value=4,
            ),
            html.Br(),
            html.Br(),
                           
            html.Div([html.Label("Genes: "),
                      dcc.RadioItems(
                          id = 'dotplot_var_type',
                          options = ['Automatic DEGs', 'Specify Genes'],
                          value = 'Automatic DEGs',
                      ),
                     ],
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            
            html.Div([html.Label("Categories: "),
                      dcc.RadioItems(
                          id = 'dotplot_obs_type',
                          options = ['Automatic order', 'Customize order'],
                          value = 'Automatic order',
                      ),
                     ],
                     style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),

            html.Div(children=[], 
                     id='dynamic_dotplot_var_type',
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            html.Div(children=[], 
                     id='dynamic_dotplot_obs_type',
                     style={'width': '40%', 'float': 'right', 'display': 'inline-block'}),
 
        ],
                             style={'width': '100%', 'float': 'left', 'display': 'inline-block'},#left tab style
                             id = 'left_tab2')
                          ],
                         style={'width': '20%', 'float': 'left', 'display': 'inline-block'},#left tab style
                         id = 'left_tab1'),

                 ####right tab
                 html.Div(children = [],
                         style={'width': '75%', 'float': 'right', 'display': 'inline-block'},#right tab style
                         id = 'right_tab1'), 
    
                ],
               )
##############################################
##########  generate left tab      ###########
##############################################
@app.callback(
    Output('left_tab1', 'children'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def generate_left_tab1(anndata):
    if anndata is not None:
        children = [html.Label("Select obs:"),
                    dcc.Dropdown(
                        id="dropdown_obs_dotplot",
                        options = list(anndata_objects[anndata].obs.dtypes[anndata_objects[anndata].obs.dtypes =='category'].index),
                    ),
                    html.Br(),
                    html.Br(),
                           
                    html.Div(children = [],
                             style={'width': '100%', 'float': 'left', 'display': 'inline-block'},#left tab style
                             id = 'left_tab2')]
    else: 
        children = []
    return children

@app.callback(
    Output('left_tab2', 'children'),
    Input('dropdown_obs_dotplot', 'value'),
    prevent_initial_call=True
)
def generate_left_tab2(dropdown_obs_dotplot):
    if dropdown_obs_dotplot is not None:
        children = [
            html.Label("Select method:"),
            dcc.Dropdown(id="dropdown_dotplot_method",
                         options = ['logreg', 't-test', 'wilcoxon', 't-test_overestim_var'], 
                         value = 'wilcoxon',
                        ),
            html.Br(),
            html.Br(),
                           
            html.Label("min fold change: "),
            dcc.Input(
                id = 'dotplot_min_log_fold_change',
                type='number',
                value=2.5,
            ),
            html.Br(),
            html.Br(),
                           
            html.Label("vmin: "),
            dcc.Input(
                id = 'dotplot_vmin',
                type='number',
                max = 0,
                value=-4,
            ),
            html.Br(),
            html.Br(),
                           
            html.Label("vmax: "),
            dcc.Input(
                id = 'dotplot_vmax',
                type='number',
                min = 0,
                value=4,
            ),
            html.Br(),
            html.Br(),
                           
            html.Div([html.Label("Genes: "),
                      dcc.RadioItems(
                          id = 'dotplot_var_type',
                          options = ['Automatic DEGs', 'Specify Genes'],
                          value = 'Automatic DEGs',
                      ),
                     ],
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            
            html.Div([html.Label("Categories: "),
                      dcc.RadioItems(
                          id = 'dotplot_obs_type',
                          options = ['Automatic order', 'Customize order'],
                          value = 'Automatic order',
                      ),
                     ],
                     style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),

            html.Div(children=[], 
                     id='dynamic_dotplot_var_type',
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            html.Div(children=[], 
                     id='dynamic_dotplot_obs_type',
                     style={'width': '40%', 'float': 'right', 'display': 'inline-block'}),
 
        ]
    else: 
        children = []
    return children
##############################################
##############################################

##############################################
##########  generate right tab     ###########
##############################################
@app.callback(
    Output('right_tab1', 'children'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def generate_righ_tab1(anndata):
    if anndata is not None:
        children = [
            html.Div(children = [],
                     id = 'dotplot_figure',
                    ),
            html.Div(children = [],
                     id = 'dotplot_button',
                    ),
                   ]
    else: 
        children = []
    return children

@app.callback(
    Output('dotplot_button', 'children'),
    Input('dropdown_obs_dotplot', 'value'),
    prevent_initial_call=True
)
def generate_dotplot_button(dropdown_obs_dotplot):
    if dropdown_obs_dotplot is not None:
        children = [html.Button('Generate Plot', id='dotplot_button_press', n_clicks=0)]
    else: 
        children = []
    return children
##############################################
##############################################

##########################################################
#################### app layout   ########################
##########################################################
app.layout = html.Div([
    ##############################################
    ####select anndata object from dictionary#####
    ##############################################
    html.Label("Select object"),
    dcc.Dropdown(options = list(anndata_objects), 
                 id="anndata",
                ),
    ##############################################

    ##############################################
    ###############tabular layout#################
    ##############################################
    dcc.Tabs([
        dcc.Tab(tab1, label="Interactive UMAP"),
        dcc.Tab(tab2, label="Dotplot",),
        ]),
    ##############################################
])
##########################################################
##########################################################


##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 1 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

@app.callback(
    Output('dropdown_var', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].var.index.tolist()

@app.callback(
    Output('dropdown_obs', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].obs.columns.tolist()

##########################################################
##########################################################

##########################################################
########## plot expression of gene on umap  ##############
##########################################################
@app.callback(
    Output("graph1", "figure"),
    Output('dropdown_var','value'),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),    
    Input("dropdown_var", "value"),
    prevent_initial_call=True
)

def update_graph_1(slider_marker_size,
                   anndata,
                   dropdown_var,
                  ):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    
    else:
        color_var = dropdown_var
        
    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = anndata_objects[anndata][:,color_var].X.toarray().flatten(),
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        color_continuous_scale = plotly_reds,
                        labels = {'color':color_var},
                        width=700, height=700,
                       )
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(uirevision='constant')
    return fig, color_var

##########################################################
##########################################################

##########################################################
########## plot annotation of obs  on umap  ##############
##########################################################
@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_graph_2(dropdown_obs, slider_marker_size,anndata):
    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[dropdown_obs]):
        try:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[dropdown_obs],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={dropdown_obs: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 2 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

# @app.callback(
#     Output('dropdown_obs_dotplot', 'options'),
#     Input('anndata', 'value'),
#     prevent_initial_call=True
# )
# def update_output(anndata):
#     return anndata_objects[anndata].obs.columns.tolist()

# @app.callback(
#     Output('dropdown_var_dotplot', 'options'),
#     Input('anndata', 'value'),
#     prevent_initial_call=True
# )
# def update_output(anndata):
#     return anndata_objects[anndata].var.index.tolist()


##########################################################
##########################################################

##########################################################
##########     Get specified  genes   ####################
##########################################################
# @app.callback(
#     Output('dd_output_container', 'children'),
#     Input('dropdown_var_dotplot', 'value')
# )
# def update_output(value):
#     return f'You have selected {value}'

##########################################################
##########################################################

##########################################################
##########     dynamic dotplot menu          #############
##########################################################
@app.callback(
    Output('dynamic_dotplot_var_type', 'children'),
    Input('dotplot_var_type','value'),
    Input('anndata', 'value'),
    Input('dropdown_obs_dotplot','value'),
    prevent_initial_call=False
)
def generate_dropdown(dotplot_var_type,anndata,dropdown_obs_dotplot):
    if dotplot_var_type == 'Automatic DEGs':
        children = [
            html.Label("n_genes: "),
            dcc.Input(
                id = 'dotplot_n_genes',
                type='number',
                min = 0,
                step = 1,
                value=4,
            ),
            html.Br(),
            html.Br(), 
            html.Label("Select categories"),
            dcc.Dropdown(
                id="dropdown_obs_dotplot",
                options = list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),
                value = list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),
                multi = True,
                ),
        ]
    elif dotplot_var_type == 'Specify Genes':
         children = [
             html.Label("Select genes"),
                      dcc.Dropdown(
                          id="dropdown_var_dotplot",
                          options = anndata_objects[anndata].var.index.tolist(),
                          multi = True,
                      ),
        ]
    else:
        children = []
    return children

@app.callback(
    Output('dynamic_dotplot_obs_type', 'children'),
    Input('dotplot_obs_type','value'),
    Input('anndata', 'value'),
    Input('dropdown_obs_dotplot','value'),
    prevent_initial_call=False
)
def generate_dropdown(dotplot_obs_type,anndata,dropdown_obs_dotplot):
    if dotplot_obs_type == 'Customize order':
        df_list = pd.DataFrame(list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),columns=['order categories:'])
        children = [dag.AgGrid(
            id="select_categories",
            rowData=df_list.to_dict("records"),
            columnDefs=[{"field": i,} for i in df_list.columns],
            dashGridOptions={
                "animateRows": True,
                "enableCellTextSelection": False,
                "ensureDomOrder": True,
                "rowDragManaged": True,
                #"rowDragMultiRow": True,
                "rowSelection": "multiple",
                "rowDragEntireRow": True,
                "pagination": False,
                "suppressRowClickSelection": True,
            },
        )
        ]
    else:
        children = []
    return children
##########################################################
##########################################################

##########################################################
############     generate dotplot         ################
##########################################################
# @app.callback(
#     Output('dotplot_figure', 'children'),
#     Output('dotplot_button_press', 'n_clicks'),
#     Input('dotplot_button_press', 'n_clicks'),
#     Input('anndata', 'value'),
#     Input('dropdown_obs_dotplot','value'),
#     Input('dropdown_dotplot_method','value'),
#     Input('dotplot_min_log_fold_change','value'),
#     Input('dotplot_vmin','value'),
#     Input('dotplot_vmax','value'),
#     Input('dotplot_var_type','value'),
#     Input('dotplot_n_genes','value'), #store
#     Input('dropdown_var_dotplot','value'), #store 
#     #Input('dropdown_obs_dotplot','value'), #store
#     Input('dotplot_obs_type','value'),

# )
# def generate_dotplot_figure(dotplot_button_press,
#                             anndata,
#                             dropdown_obs_dotplot,
#                             dropdown_dotplot_method,
#                             dotplot_min_log_fold_change,
#                             dotplot_vmin,
#                             dotplot_vmax,
#                             dotplot_var_type,
#                             dotplot_n_genes,
#                             dropdown_var_dotplot,
#                             dotplot_obs_type,
#                            ):
#     if dotplot_button_press > 0:
#         sc.tl.rank_genes_groups(anndata_objects[anndata], 
#                                 groupby = dropdown_obs_dotplot,
#                                 method = dropdown_dotplot_method,
#                                 key_added = dropdown_dotplot_method+'_'+dropdown_obs_dotplot,
#                                )
        
#         fig_buffer = BytesIO()

#         dp = sc.pl.rank_genes_groups_dotplot(anndata_objects[anndata], 
#                                 groupby = dropdown_obs_dotplot,
#                                 n_genes=dotplot_n_genes,
#             values_to_plot="logfoldchanges",
#             cmap='bwr',
#             vmin=dotplot_vmin,
#             vmax=dotplot_vmax,
#             min_logfoldchange=dotplot_min_log_fold_change,
#                                         key = dropdown_dotplot_method+'_'+dropdown_obs_dotplot,
#             colorbar_title='log fold change',
#                                        return_fig = True)
#         dp.savefig(fig_buffer, format='png')
#         plt.close()
#         encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")

#         fig_src = "data:image/png;base64, " + encoded
#         children = [html.Img(id="dotplot",className="image", src = fig_src, 
#                              style={'width': '100%', 'float': 'right', 'display': 'inline-block'})]
        
#         n_clicks = 0
#     else: 
#         children = []
#         n_clicks = 0
#     return children,n_clicks
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

if __name__ == '__main__':
    app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=True)
print('Use this link: '+'http://cranio213:'+str(port))

In [ ]:
port = 11199
initial_gene = 'Car4'
initial_obs = 'sample'

app = Dash(__name__,suppress_callback_exceptions=True)


#####selection list grid
df_list = pd.DataFrame(list(anndata_objects['ecto_interactive_nocycle'].obs['leiden1'].cat.categories),columns=['order categories:'])
grid = dag.AgGrid(
    id="select_categories",
    rowData=df_list.to_dict("records"),
    columnDefs=[{"field": i,} for i in df_list.columns],
    dashGridOptions={
                "animateRows": True,
                "enableCellTextSelection": False,
                "ensureDomOrder": True,
                "rowDragManaged": True,
                #"rowDragMultiRow": True,
                "rowSelection": "multiple",
                "rowDragEntireRow": True,
                "pagination": False,
                "suppressRowClickSelection": True,
            },
)

##########################################################
######################  Display of tab 1 #################
##########################################################
tab1 = html.Div([
    html.Div([
        html.Div([
            
            
            ##############################################
            
            ##############################################
            ######  select gene to plot from list  #######
            ##############################################
            html.Label("Select gene"),
            dcc.Dropdown(
                         id="dropdown_var",
                        value = initial_gene,
                        ),
            ##############################################
            
            ##############################################
            ####other gene values to show if available####
            ##############################################
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            ##############################################
            
            ##############################################
            #######set dot size of both umap plots########
            ##############################################
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
            ##############################################
        ],
            style={'width': '49%', 'display': 'inline-block'}), #html style of left dropdown section

        
        html.Div([
            ##############################################
            ######  select obs to plot from list  #######
            ##############################################
            html.Label("Select obs"),
            dcc.Dropdown(
                         id="dropdown_obs",
                         value = initial_obs
                        ),
            ##############################################
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}), #html style of right dropdown section
        
    ],
        style={'padding': '10px 5px'}), #html style of dropdown section

    ##############################################
    ##########  plot left umap graph  ############
    ##############################################
    html.Div([
      dcc.Graph(id="graph1"),
        ],
        style={'width': '49%', 'display': 'inline-block'}),#html style of left graph section
    ##############################################
    
    ##############################################
    ##########  plot right umap graph  ###########
    ##############################################
    html.Div([
      dcc.Graph(id="graph2")
    ],
            style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),#html style of right graph section
    ##############################################
    
    html.Div(dcc.Store(id='gene_plotted')),
])


##########################################################
######################  Display of tab 2 #################
##########################################################
tab2 = html.Div([
                ###left tab
                 html.Div(children = [],
                         style={'width': '20%', 'float': 'left', 'display': 'inline-block'},#left tab style
                         id = 'left_tab1'),

                 ####right tab
                 html.Div(children = [],
                         style={'width': '75%', 'float': 'right', 'display': 'inline-block'},#right tab style
                         id = 'right_tab1'), 
    
    ##############################################
    ############data stores for vars##############
    ##############################################
    html.Div(dcc.Store(id='dotplot_n_genes_store',data = None)),
    html.Div(dcc.Store(id='dropdown_var_dotplot_store',data = None)),
    html.Div(dcc.Store(id='dropdown_obs_dotplot_store',data = None)),
    ##############################################
                ],
               )
##############################################
##########  generate left tab      ###########
##############################################
@app.callback(
    Output('left_tab1', 'children'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def generate_left_tab1(anndata):
    if anndata is not None:
        children = [html.Label("Select obs:"),
                    dcc.Dropdown(
                        id="dropdown_obs_dotplot",
                        options = list(anndata_objects[anndata].obs.dtypes[anndata_objects[anndata].obs.dtypes =='category'].index),
                    ),
                    html.Br(),
                    html.Br(),
                           
                    html.Div(children = [],
                             style={'width': '100%', 'float': 'left', 'display': 'inline-block'},#left tab style
                             id = 'left_tab2')]
    else: 
        children = []
    return children

@app.callback(
    Output('left_tab2', 'children'),
    Input('dropdown_obs_dotplot', 'value'),
    prevent_initial_call=True
)
def generate_left_tab2(dropdown_obs_dotplot):
    if dropdown_obs_dotplot is not None:
        children = [
            html.Label("Select method:"),
            dcc.Dropdown(id="dropdown_dotplot_method",
                         options = ['logreg', 't-test', 'wilcoxon', 't-test_overestim_var'], 
                         value = 'wilcoxon',
                        ),
            html.Br(),
            html.Br(),
                           
            html.Label("min fold change: "),
            dcc.Input(
                id = 'dotplot_min_log_fold_change',
                type='number',
                value=2.5,
            ),
            html.Br(),
            html.Br(),
                           
            html.Label("vmin: "),
            dcc.Input(
                id = 'dotplot_vmin',
                type='number',
                max = 0,
                value=-4,
            ),
            html.Br(),
            html.Br(),
                           
            html.Label("vmax: "),
            dcc.Input(
                id = 'dotplot_vmax',
                type='number',
                min = 0,
                value=4,
            ),
            html.Br(),
            html.Br(),
                           
            html.Div([html.Label("Genes: "),
                      dcc.RadioItems(
                          id = 'dotplot_var_type',
                          options = ['Automatic DEGs', 'Specify Genes'],
                          value = 'Automatic DEGs',
                      ),
                     ],
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            
            html.Div([html.Label("Categories: "),
                      dcc.RadioItems(
                          id = 'dotplot_obs_type',
                          options = ['Automatic order', 'Customize order'],
                          value = 'Automatic order',
                      ),
                     ],
                     style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),

            html.Div(children=[], 
                     id='dynamic_dotplot_var_type',
                     style={'width': '49%', 'float': 'left', 'display': 'inline-block'}),
            html.Div(children=[], 
                     id='dynamic_dotplot_obs_type',
                     style={'width': '40%', 'float': 'right', 'display': 'inline-block'}),
 
        ]
    else: 
        children = []
    return children
##############################################
##############################################

##############################################
##########  generate right tab     ###########
##############################################
@app.callback(
    Output('right_tab1', 'children'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def generate_righ_tab1(anndata):
    if anndata is not None:
        children = [
            html.Div(children = [],
                     id = 'dotplot_figure',
                    ),
            html.Div(children = [],
                     id = 'dotplot_button',
                    ),
                   ]
    else: 
        children = []
    return children

@app.callback(
    Output('dotplot_button', 'children'),
    Input('dropdown_obs_dotplot', 'value'),
    prevent_initial_call=True
)
def generate_dotplot_button(dropdown_obs_dotplot):
    if dropdown_obs_dotplot is not None:
        children = [html.Button('Generate Plot', id='dotplot_button_press', n_clicks=0)]
    else: 
        children = []
    return children
##############################################
##############################################

##########################################################
#################### app layout   ########################
##########################################################
app.layout = html.Div([
    ##############################################
    ####select anndata object from dictionary#####
    ##############################################
    html.Label("Select object"),
    dcc.Dropdown(options = list(anndata_objects), 
                 id="anndata",
                ),
    ##############################################

    ##############################################
    ###############tabular layout#################
    ##############################################
    dcc.Tabs([
        dcc.Tab(tab1, label="Interactive UMAP"),
        dcc.Tab(tab2, label="Dotplot",),
        ]),
    ##############################################
])
##########################################################
##########################################################


##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 1 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

@app.callback(
    Output('dropdown_var', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].var.index.tolist()

@app.callback(
    Output('dropdown_obs', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return anndata_objects[anndata].obs.columns.tolist()

##########################################################
##########################################################

##########################################################
########## plot expression of gene on umap  ##############
##########################################################
@app.callback(
    Output("graph1", "figure"),
    Output('dropdown_var','value'),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),    
    Input("dropdown_var", "value"),
    prevent_initial_call=True
)

def update_graph_1(slider_marker_size,
                   anndata,
                   dropdown_var,
                  ):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    
    else:
        color_var = dropdown_var
        
    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = anndata_objects[anndata][:,color_var].X.toarray().flatten(),
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        color_continuous_scale = plotly_reds,
                        labels = {'color':color_var},
                        width=700, height=700,
                       )
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(uirevision='constant')
    return fig, color_var

##########################################################
##########################################################

##########################################################
########## plot annotation of obs  on umap  ##############
##########################################################
@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_graph_2(dropdown_obs, slider_marker_size,anndata):
    tmp_df = pd.DataFrame(anndata_objects[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = anndata_objects[anndata].obs.index
    tmp_df = tmp_df.join(anndata_objects[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[dropdown_obs]):
        try:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[dropdown_obs],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={dropdown_obs: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 2 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

# @app.callback(
#     Output('dropdown_obs_dotplot', 'options'),
#     Input('anndata', 'value'),
#     prevent_initial_call=True
# )
# def update_output(anndata):
#     return anndata_objects[anndata].obs.columns.tolist()

# @app.callback(
#     Output('dropdown_var_dotplot', 'options'),
#     Input('anndata', 'value'),
#     prevent_initial_call=True
# )
# def update_output(anndata):
#     return anndata_objects[anndata].var.index.tolist()


##########################################################
##########################################################

##########################################################
##########     Get specified  genes   ####################
##########################################################
# @app.callback(
#     Output('dd_output_container', 'children'),
#     Input('dropdown_var_dotplot', 'value')
# )
# def update_output(value):
#     return f'You have selected {value}'

##########################################################
##########################################################

##########################################################
##########     dynamic dotplot menu          #############
##########################################################
@app.callback(
    Output('dynamic_dotplot_var_type', 'children'),
    Input('dotplot_var_type','value'),
    Input('anndata', 'value'),
    Input('dropdown_obs_dotplot','value'),
    prevent_initial_call=False
)
def generate_dropdown(dotplot_var_type,anndata,dropdown_obs_dotplot):
    if dotplot_var_type == 'Automatic DEGs':
        children = [
            html.Label("n_genes: "),
            dcc.Input(
                id = 'dotplot_n_genes',
                type='number',
                min = 0,
                step = 1,
                value=4,
            ),
            html.Br(),
            html.Br(), 
            html.Label("Select categories"),
            dcc.Dropdown(
                id="dropdown_obs_dotplot",
                options = list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),
                value = list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),
                multi = True,
                ),
        ]
    elif dotplot_var_type == 'Specify Genes':
         children = [
             html.Label("Select genes"),
                      dcc.Dropdown(
                          id="dropdown_var_dotplot",
                          options = anndata_objects[anndata].var.index.tolist(),
                          multi = True,
                      ),
        ]
    else:
        children = []
    return children

@app.callback(
    Output('dynamic_dotplot_obs_type', 'children'),
    Input('dotplot_obs_type','value'),
    Input('anndata', 'value'),
    Input('dropdown_obs_dotplot','value'),
    prevent_initial_call=False
)
def generate_dropdown(dotplot_obs_type,anndata,dropdown_obs_dotplot):
    if dotplot_obs_type == 'Customize order':
        df_list = pd.DataFrame(list(anndata_objects[anndata].obs[dropdown_obs_dotplot].cat.categories),columns=['order categories:'])
        children = [dag.AgGrid(
            id="select_categories",
            rowData=df_list.to_dict("records"),
            columnDefs=[{"field": i,} for i in df_list.columns],
            dashGridOptions={
                "animateRows": True,
                "enableCellTextSelection": False,
                "ensureDomOrder": True,
                "rowDragManaged": True,
                #"rowDragMultiRow": True,
                "rowSelection": "multiple",
                "rowDragEntireRow": True,
                "pagination": False,
                "suppressRowClickSelection": True,
            },
        )
        ]
    else:
        children = []
    return children
##########################################################
##########################################################

##########################################################
############     generate dotplot         ################
##########################################################
# @app.callback(
#     Output('dotplot_figure', 'children'),
#     Output('dotplot_button_press', 'n_clicks'),
#     Input('dotplot_button_press', 'n_clicks'),
#     Input('anndata', 'value'),
#     Input('dropdown_obs_dotplot','value'),
#     Input('dropdown_dotplot_method','value'),
#     Input('dotplot_min_log_fold_change','value'),
#     Input('dotplot_vmin','value'),
#     Input('dotplot_vmax','value'),
#     Input('dotplot_var_type','value'),
#     Input('dotplot_n_genes','value'), #store
#     Input('dropdown_var_dotplot','value'), #store 
#     #Input('dropdown_obs_dotplot','value'), #store
#     Input('dotplot_obs_type','value'),

# )
# def generate_dotplot_figure(dotplot_button_press,
#                             anndata,
#                             dropdown_obs_dotplot,
#                             dropdown_dotplot_method,
#                             dotplot_min_log_fold_change,
#                             dotplot_vmin,
#                             dotplot_vmax,
#                             dotplot_var_type,
#                             dotplot_n_genes,
#                             dropdown_var_dotplot,
#                             dotplot_obs_type,
#                            ):
#     if dotplot_button_press > 0:
#         sc.tl.rank_genes_groups(anndata_objects[anndata], 
#                                 groupby = dropdown_obs_dotplot,
#                                 method = dropdown_dotplot_method,
#                                 key_added = dropdown_dotplot_method+'_'+dropdown_obs_dotplot,
#                                )
        
#         fig_buffer = BytesIO()

#         dp = sc.pl.rank_genes_groups_dotplot(anndata_objects[anndata], 
#                                 groupby = dropdown_obs_dotplot,
#                                 n_genes=dotplot_n_genes,
#             values_to_plot="logfoldchanges",
#             cmap='bwr',
#             vmin=dotplot_vmin,
#             vmax=dotplot_vmax,
#             min_logfoldchange=dotplot_min_log_fold_change,
#                                         key = dropdown_dotplot_method+'_'+dropdown_obs_dotplot,
#             colorbar_title='log fold change',
#                                        return_fig = True)
#         dp.savefig(fig_buffer, format='png')
#         plt.close()
#         encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")

#         fig_src = "data:image/png;base64, " + encoded
#         children = [html.Img(id="dotplot",className="image", src = fig_src, 
#                              style={'width': '100%', 'float': 'right', 'display': 'inline-block'})]
        
#         n_clicks = 0
#     else: 
#         children = []
#         n_clicks = 0
#     return children,n_clicks
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

if __name__ == '__main__':
    app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=True)
print('Use this link: '+'http://cranio213:'+str(port))

In [ ]:
##Dotplot requirements
#automatic dotplot with n genes for i clustering
#specific genes in order for i clustering
#specific order of i clustering

from dash import Dash, html

from io import BytesIO

import base64

import matplotlib.pyplot as plt


app = Dash(__name__)

fig_buffer = BytesIO()

dp = sc.pl.rank_genes_groups_dotplot(anndata_objects['ecto_interactive_nocycle'], 
                                groupby = 'leiden1',
                                n_genes=4,
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-4,
    vmax=4,
    min_logfoldchange=2,
                                key = 'wilcoxon_leiden1',
    colorbar_title='log fold change',
                               return_fig = True)
dp.savefig(fig_buffer, format='png')
plt.close()
encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")

fig_src = "data:image/png;base64, " + encoded


app.layout = html.Div([html.Img(id="static_heatmap",className="image", src = fig_src)])


if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from dash import Dash, html

from io import BytesIO

import base64

import matplotlib.pyplot as plt

fig_buffer = BytesIO()
dp = sc.pl.rank_genes_groups_dotplot(anndata_objects['ecto_interactive_nocycle'], 
                                groupby = 'leiden1',
                                n_genes=4,
                                     dendrogram = False,
                                     groups = ['1','3','8'],
                                     categories_order = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','0'],
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-4,
    vmax=4,
    min_logfoldchange=2,
                                key = 'wilcoxon_leiden1',
    colorbar_title='log fold change',
                               return_fig = True)
dp.savefig(fig_buffer, format='png')    

In [ ]:
fig_buffer

In [ ]:
#Interactive dash app to view 3D plot
#create quick dataframe
tmp_df = pd.DataFrame(adata.obsm['X_umap'], columns = ['X','Y'])
tmp_df.index = adata.obs.index
tmp_df = tmp_df.join(adata.obs)
tmp_df['barcode'] = list(tmp_df.index) 
port = 11188
app = Dash(__name__)


app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label("Select gene"),
            dcc.Dropdown(adata.var.index.tolist(), 
                         value='Car4',
                         id="dropdown_var",
                        ),
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
        ],
            style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Select obs"),
            dcc.Dropdown(adata.obs.columns.tolist(), 
                         value='leiden1',
                         id="dropdown_obs",
                        ),
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),
    ],
        style={'padding': '10px 5px'}),
    html.Div([
      dcc.Graph(id="graph1"),
        
    ],
        style={'width': '49%', 'display': 'inline-block'}),
     html.Div([
      dcc.Graph(id="graph2")
    ],
        style={'width': '49%', 'display': 'inline-block'}),
    html.Div(id="cluster_select",
        style={'width': '49%', 'display': 'inline-block'},),
    html.Div(id='deg_table'),
    html.Div(dcc.Store(id='gene_plotted')),
])
    
                
            

@app.callback(
    Output("graph1", "figure"),
    Output('gene_plotted','data'),
    Input("slider_marker_size", "value"),
    Input('deg_table_callback', 'active_cell'),
    Input("dropdown_var", "value"),
    Input('deg_table_callback', 'derived_viewport_data'),
)
def update_graph_1(size_var,active_cell,dropdown_var,data_deg):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    elif callback_id == 'deg_table_callback':
        try:
            color_var = data_deg[active_cell['row']]['names']
        except:
            color_var = dropdown_var
        
    else:
        color_var = dropdown_var
    plot_df = tmp_df
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = adata[:,color_var].X.toarray().flatten(),
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        color_continuous_scale = plotly_reds,
                        labels = {'color':color_var},
                        width=700, height=700,
                       )
    fig.update_traces(marker={'size': size_var,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(uirevision='constant')
    return fig, color_var

@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
)
def update_graph_2(color_var, size_var):
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[color_var]):
        try:
            sorted_list = sorted(list(plot_df[color_var].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[color_var].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[color_var],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={color_var: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': size_var,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig


@app.callback(
    Output("gene_id", "children"),
    Input("gene_plotted", "data"),
)
def update_gene_id(gene_selected):
    gene_id = adata.var.loc[gene_selected,'gene_symbol']
    return f'{gene_id}'



@app.callback(Output('cluster_select','children'),
              Input('dropdown_obs','value'),
             )
def cluster_dropdown(leiden_res):
    if 'leiden' in leiden_res or 'annotation_paper_coarse' in leiden_res or 'annotation_2024' in leiden_res:
        return dcc.Dropdown(list(adata.obs[leiden_res].cat.categories), 
                           value=list(adata.obs[leiden_res].cat.categories)[0],id = 'cluster_select2')

@app.callback(Output('deg_table','children'),
              Input('dropdown_obs','value'),
              Input('cluster_select2','value'),
             )
def update_datatable(leiden_res,cluster_select):            
    if 'leiden' in leiden_res or 'annotation_paper_coarse' in leiden_res or 'annotation_2024' in leiden_res:
        deg_df = sc.get.rank_genes_groups_df(adata, group=None, key='wilcoxon_'+leiden_res)
        deg_df = deg_df[deg_df['group']==cluster_select].iloc[:,1:6]
        data_deg = deg_df.to_dict('records')
        columns =  [{"name": i, "id": i,} for i in (deg_df.columns)]
        return dash_table.DataTable(data=data_deg, 
                                    columns=columns,
                                    filter_action="native",
                                    sort_action="native",
                                    sort_mode="multi",
                                    column_selectable="single",
                                    page_action="native",
                                    page_current= 0,
                                    page_size= 10,
                                    id = 'deg_table_callback',
                                   )

app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=False)
print('Use this link: '+'http://cranio213:'+str(port))